<h5> 
   <header>     
   <font size=10>
   <font color=green>    
   <b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  Sentiment Analysis </b>
       

In [2]:
import os 
import pandas as pd 
import numpy as np 
from time import time
import multiprocessing as mp
from multiprocessing import Process, current_process 



In [3]:
print("Number of processors: ", mp.cpu_count())


samsungvarimp =([20,  1,  1,  1,  1,  6,  1, 29, 11, 28,  1, 10, 25, 17,  1,  1,  5,
       30,  2, 16,  1,  3, 21,  7,  1,  1,  1, 23,  8,  9, 12,  1, 19,  1,
        1, 13,  1, 22,  1,  1,  1,  1, 26,  1,  1,  1, 18, 24,  1,  1,  1,
        1, 15,  4, 27,  1, 14,  1])





varimpihpone = ([21,  1,  1,  1,  1, 12,  1, 33,  9, 25,  1,  7, 31,  1,  1, 34,  6,
       32, 20, 15,  1, 16, 22, 14,  1, 28,  1, 26,  2, 13,  1,  1, 19,  8,
        1, 10,  1, 23,  3,  1,  1,  1, 30,  1,  1, 11, 27, 24,  1,  1,  1,
        1, 18,  1, 29,  5, 17,  4])


Number of processors:  4


In [39]:
samsung_df = pd.read_csv(r"galaxy_smallmatrix_labeled_9d.csv")
iphone_df = pd.read_csv(r"iphone_smallmatrix_labeled_9d.csv")




In [5]:
def cut_cols(df,x):
    df = df.copy(deep=True)
    cols = [col for col in df if col.startswith(x) ]
    df = df[cols]
    return df 
    

In [6]:
iphone_cut = cut_cols(iphone_df,'i')
samsung_cut  = cut_cols(samsung_df,'sam')

samsung_cut['galaxysentiment'] = samsung_df['galaxysentiment']


In [7]:
iphone_df.hist(column='iphonesentiment');
samsung_df.hist(column='galaxysentiment');

In [8]:
print(iphone_df.describe())

samsung_df.columns

             iphone  samsunggalaxy    sonyxperia   nokialumina      htcphone  \
count  12973.000000   12973.000000  12973.000000  12973.000000  12973.000000   
mean       2.148077       0.071148      0.024050      0.002312      0.137131   
std        5.664226       0.280869      0.171266      0.051144      4.223547   
min        0.000000       0.000000      0.000000      0.000000      0.000000   
25%        1.000000       0.000000      0.000000      0.000000      0.000000   
50%        1.000000       0.000000      0.000000      0.000000      0.000000   
75%        1.000000       0.000000      0.000000      0.000000      0.000000   
max       46.000000       8.000000      8.000000      2.000000    479.000000   

                ios  googleandroid  iphonecampos  samsungcampos    sonycampos  \
count  12973.000000   12973.000000  12973.000000   12973.000000  12973.000000   
mean       0.152316       0.039621      0.289601       0.053727      0.009944   
std        0.839081       0.255020  

Index(['iphone', 'samsunggalaxy', 'sonyxperia', 'nokialumina', 'htcphone',
       'ios', 'googleandroid', 'iphonecampos', 'samsungcampos', 'sonycampos',
       'nokiacampos', 'htccampos', 'iphonecamneg', 'samsungcamneg',
       'sonycamneg', 'nokiacamneg', 'htccamneg', 'iphonecamunc',
       'samsungcamunc', 'sonycamunc', 'nokiacamunc', 'htccamunc',
       'iphonedispos', 'samsungdispos', 'sonydispos', 'nokiadispos',
       'htcdispos', 'iphonedisneg', 'samsungdisneg', 'sonydisneg',
       'nokiadisneg', 'htcdisneg', 'iphonedisunc', 'samsungdisunc',
       'sonydisunc', 'nokiadisunc', 'htcdisunc', 'iphoneperpos',
       'samsungperpos', 'sonyperpos', 'nokiaperpos', 'htcperpos',
       'iphoneperneg', 'samsungperneg', 'sonyperneg', 'nokiaperneg',
       'htcperneg', 'iphoneperunc', 'samsungperunc', 'sonyperunc',
       'nokiaperunc', 'htcperunc', 'iosperpos', 'googleperpos', 'iosperneg',
       'googleperneg', 'iosperunc', 'googleperunc', 'galaxysentiment'],
      dtype='object')

<h5>
    <header> 
        <font size = 10 >
            <font color = green>
                &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Missing Data 

In [9]:
def missing_percentage_data(df):
    
    
    missing_values_count = df.isnull().sum()

    total_cells = np.product(df.shape)
    
    total_missing = missing_values_count.sum()

    missing_percent = (total_missing/total_cells) * 100

    print('Percent of missing data of = {}%'.format(missing_percent))
    
    return 

In [10]:
missing_percentage_data(samsung_df)
missing_percentage_data(iphone_df)

Percent of missing data of = 0.0%
Percent of missing data of = 0.0%


In [11]:
def correlation(df, threshold):
    df = df.copy(deep=True)
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in df.columns:
                    del df[colname] # deleting the column from the dataset

    return df

In [12]:
iphone_df_corr = correlation(iphone_df,0.85)
samsung_df_corr = correlation(samsung_df,0.85)


In [13]:
def near_zero_var(df,tresh):
    df = df.copy(deep=True)
    cols = df.columns
    for i in range(len(cols)):
        if df[cols[i]].var() <= tresh :
            del df[cols[i]] 
    
    return df 
    

In [14]:
iphone_df_var = near_zero_var(iphone_df,1)
samsung_df_var = near_zero_var(samsung_df,1)

In [15]:
X_iphone= iphone_df.loc[:,iphone_df.columns != 'iphonesentiment']
Y_iphone = iphone_df['iphonesentiment']


In [ ]:
import time 
from multiprocessing import Pool 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR


start_time= time.time()
p = Pool(4)


estimator = SVR(kernel="linear")

selector = RFECV(estimator, step=1, cv=5)

selector = selector.fit(X_iphone, Y_iphone)

selector.support_ 

selector.ranking_ 



p.close()

p.join()

end_time =  time.time() - start_time 

print("--- Run Time is : %s mins--- " % np.round(((time.time()-start_time)/60),2))



Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zordo/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/zordo/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/zordo/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/zordo/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zordo/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zordo/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/zordo/anaconda3/lib/python3.7

In [ ]:
print(selector.support_)
print("\n")
print(selector.ranking_) 
print("\n")
print(selector.grid_scores_)

In [ ]:
X_samsung= samsung_df.loc[:,samsung_df.columns != 'galaxysentiment']
Y_samsung = samsung_df['galaxysentiment']


In [ ]:
import time 
from multiprocessing import Pool 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR


start_time= time.time()
p = Pool(4)


estimator = SVR(kernel="linear")

selector = RFECV(estimator, step=1, cv=5)

selector = selector.fit(X_samsung, Y_samsung)

selector.support_ 

selector.ranking_ 



p.close()

p.join()

end_time =  time.time() - start_time 

print("--- Run Time is : %s mins--- " % np.round(((time.time()-start_time)/60),2))

In [16]:
def most_imp_vars(df,varimparray):
    df_copy= df.copy()
    for i in varimparray:
        if varimparray[i] != 1:
            print(df_copy.columns[i])
            del df_copy[df_copy.columns[i]]
            
    return df_copy 
    

In [ ]:
varimp_iphone = selector.ranking_ 
test = most_imp_vars(iphone_df,varimp_iphone)

In [17]:
def X_Y_split(df,label):
    
    X= df.loc[:,df.columns != label ]
    y = df[label]
    
    return X,y 

    
    

In [42]:
X_iphone,y_iphone = X_Y_split(iphone_df,'iphonesentiment')

In [43]:
#Create training and testing sets with a 70/30 split using createDataPartition.
from sklearn.model_selection import train_test_split

X_iphone,y_iphone = X_Y_split(iphone_df,'iphonesentiment')

X_train_iphone , X_test_iphone , y_train_iphone , y_test_iphone =  train_test_split(X_iphone, y_iphone, test_size=0.33, random_state=42)




In [30]:
weights = {0:1,1:5,2:2,3:2,4:2,5:1}


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,cohen_kappa_score
import time

start_time = time.time()

rf = RandomForestClassifier()

rf.fit(X_train_iphone,y_train_iphone)

predictions = rf.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.02 mins--- 
 
Accuracy :  0.9142923867351704


Kappa's :  0.631806682189497


Confusion Matrix : 
 
   [[ 384   22]
 [ 345 3531]]


In [23]:
import xgboost as xgb 

5    2500
0     615
4     492
3     399
2     162
1     114
Name: iphonesentiment, dtype: int64

In [212]:
from sklearn.svm import SVC

start_time = time.time()

svc = SVC()

svc.fit(X_train_iphone,y_train_iphone)

predictions = svc.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.17 mins--- 
 
Accuracy :  0.7223260158804297


Kappa's :  0.44147337960791777


Confusion Matrix : 
 
   [[ 392    0    0   19   16   48]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]
 [   2    1   20  119    2    0]
 [   1    0    1    1  135    5]
 [ 220  113  141  260  339 2447]]


In [213]:
from sklearn.neighbors import KNeighborsClassifier

start_time = time.time()

knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train_iphone,y_train_iphone)

predictions = knn.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.05 mins--- 
 
Accuracy :  0.708780943484353


Kappa's :  0.47335990046448917


Confusion Matrix : 
 
   [[ 381    2    5    7   18   77]
 [   9    2    4    2    9   66]
 [   0    1   22    0    4   18]
 [   3    2    3  273    7   27]
 [  19    6    8    4  153  108]
 [ 203  101  120  113  301 2204]]


In [214]:
from sklearn import tree


start_time = time.time()

t =tree.DecisionTreeClassifier()

t.fit(X_train_iphone,y_train_iphone)

predictions = t.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.0 mins--- 
 
Accuracy :  0.7419430172816441


Kappa's :  0.5164958304546692


Confusion Matrix : 
 
   [[ 386    1    6    2   12   26]
 [   3    0    0    1    3   17]
 [   2    0   20    1    6   26]
 [   3    0    2  273    3   31]
 [   8    3    3    6  163   65]
 [ 213  110  131  116  305 2335]]


In [206]:
y_test.value_counts()

5    2500
0     615
4     492
3     399
2     162
1     114
Name: iphonesentiment, dtype: int64

In [218]:
iphone_new_df = most_imp_vars(iphone_df,varimpihpone)

htccamunc
iphonecamneg
nokiadisunc
sonycampos
samsungdisneg
iphonecampos
samsungperpos
samsungcamunc
nokiacamunc
sonydisneg
iphoneperpos
nokiadispos
nokiacampos
samsungdisunc
nokiacamneg
iphoneperneg
iphonedisneg
htcperneg
ios
nokiadisneg


In [226]:
iphone_new_df.var()



iphone             32.083452
samsunggalaxy       0.078888
sonyxperia          0.029332
nokialumina         0.002616
htcphone           17.838348
googleandroid       0.065035
samsungcampos       0.505978
htccampos           2.559563
samsungcamneg       0.281155
sonycamneg          0.007241
htccamneg           0.484882
iphonecamunc        1.546034
sonycamunc          0.005848
iphonedispos        8.411451
samsungdispos       0.588907
sonydispos          0.113962
htcdispos          18.674279
htcdisneg           0.816487
iphonedisunc        4.691716
sonydisunc          0.022104
htcdisunc           0.424081
sonyperpos          0.041399
nokiaperpos         0.105191
htcperpos           1.351032
samsungperneg       0.492032
sonyperneg          0.093160
nokiaperneg         0.093867
iphoneperunc        1.257752
samsungperunc       0.159162
sonyperunc          0.007549
nokiaperunc         0.033580
htcperunc           0.688301
iosperpos           0.303310
googleperpos        0.181987
iosperneg     

In [234]:
X_iphone,y_iphone = X_Y_split(iphone_new_df,'iphonesentiment')

X_train_iphone , X_test_iphone , y_train_iphone , y_test_iphone =  train_test_split(X_iphone, y_iphone, test_size=0.33, random_state=42)




In [236]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,cohen_kappa_score
import time

start_time = time.time()

rf = RandomForestClassifier()

rf.fit(X_train_iphone,y_train_iphone)

predictions = rf.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.01 mins--- 
 
Accuracy :  0.7428771602055114


Kappa's :  0.4955686480781568


Confusion Matrix : 
 
   [[ 385    0    0    3    7    8]
 [   1    1    0    0    0    2]
 [   1    0   20    1    1    5]
 [   7    0    2  202    7   35]
 [   4    0    0    3  145   22]
 [ 217  113  140  190  332 2428]]


In [243]:
iphone_new_df2 = correlation(iphone_df, 0.85)

In [244]:
X_iphone,y_iphone = X_Y_split(iphone_new_df2,'iphonesentiment')

X_train_iphone , X_test_iphone , y_train_iphone , y_test_iphone =  train_test_split(X_iphone, y_iphone, test_size=0.33, random_state=42)




In [245]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,cohen_kappa_score
import time

start_time = time.time()

rf = RandomForestClassifier()

rf.fit(X_train_iphone,y_train_iphone)

predictions = rf.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.01 mins--- 
 
Accuracy :  0.7510509107893508


Kappa's :  0.5160689884536862


Confusion Matrix : 
 
   [[ 385    1    1    4    6   11]
 [   0    0    0    0    0    2]
 [   1    0   19    1    0    9]
 [   5    1    1  238    5   31]
 [   4    1    1    2  151   24]
 [ 220  111  140  154  330 2423]]


In [242]:
iphone_new_df3 = near_zero_var(iphone_df,1)

In [248]:
X_iphone,y_iphone = X_Y_split(iphone_new_df3,'iphonesentiment')

X_train_iphone , X_test_iphone , y_train_iphone , y_test_iphone =  train_test_split(X_iphone, y_iphone, test_size=0.33, random_state=42)




In [249]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,cohen_kappa_score
import time

start_time = time.time()

rf = RandomForestClassifier()

rf.fit(X_train_iphone,y_train_iphone)

predictions = rf.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.01 mins--- 
 
Accuracy :  0.7340028024287716


Kappa's :  0.4707119833994583


Confusion Matrix : 
 
   [[ 386    2   20    3    8   13]
 [   1    0    0    0    0    2]
 [   0    0    3    0    1    8]
 [   1    0    0  158    0   10]
 [   2    0    0    4  156   27]
 [ 225  112  139  234  327 2440]]


In [252]:
iphone_new_df = most_imp_vars(iphone_df,varimpihpone)
iphone_new_df = correlation(iphone_new_df, 0.85)
iphone_new_df = near_zero_var(iphone_new_df,1)

htccamunc
iphonecamneg
nokiadisunc
sonycampos
samsungdisneg
iphonecampos
samsungperpos
samsungcamunc
nokiacamunc
sonydisneg
iphoneperpos
nokiadispos
nokiacampos
samsungdisunc
nokiacamneg
iphoneperneg
iphonedisneg
htcperneg
ios
nokiadisneg


In [255]:
X_iphone,y_iphone = X_Y_split(iphone_new_df,'iphonesentiment')

X_train_iphone , X_test_iphone , y_train_iphone , y_test_iphone =  train_test_split(X_iphone, y_iphone, test_size=0.33, random_state=42)




In [256]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,cohen_kappa_score
import time

start_time = time.time()

rf = RandomForestClassifier()

rf.fit(X_train_iphone,y_train_iphone)

predictions = rf.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.01 mins--- 
 
Accuracy :  0.7048108360579168


Kappa's :  0.38684228503107065


Confusion Matrix : 
 
   [[ 385    0   20    5    6    7]
 [   0    0    0    0    0    1]
 [   2    0    0    1    0    7]
 [   0    0    0   16    3    3]
 [   4    0    0    2  146   11]
 [ 224  114  142  375  337 2471]]


In [257]:
y_test.value_counts()

5    2500
0     615
4     492
3     399
2     162
1     114
Name: iphonesentiment, dtype: int64

In [258]:
iphone_df['iphonesentiment'].value_counts()

5    7540
0    1962
4    1439
3    1188
2     454
1     390
Name: iphonesentiment, dtype: int64

In [267]:
minority = len(iphone_df[iphone_df['iphonesentiment'] == 1])

In [274]:
iphone_df[iphone_df['iphonesentiment'] == 1].index

Int64Index([ 824,  825,  827,  828,  829,  830,  831,  832,  833,  834,
            ...
            1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1306, 1308],
           dtype='int64', length=390)

In [289]:

dic = { 0: 0, 1:0, 2:0, 3:0, 4:0, 5:0 }

for i in range(6):
    dic[i]= iphone_df[iphone_df['iphonesentiment'] == i].index
    print(dic[i])
    

Int64Index([    0,     1,     2,     3,     4,     7,     8,     9,    11,
               12,
            ...
            12867, 12871, 12885, 12906, 12914, 12919, 12923, 12934, 12950,
            12969],
           dtype='int64', length=1962)
Int64Index([ 824,  825,  827,  828,  829,  830,  831,  832,  833,  834,
            ...
            1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1306, 1308],
           dtype='int64', length=390)
Int64Index([  103,   154,   310,   333,   450,   550,   591,   666,   884,
             1028,
            ...
            10926, 11099, 11132, 11397, 11711, 11887, 11973, 12000, 12442,
            12726],
           dtype='int64', length=454)
Int64Index([   10,    71,    74,   109,   138,   146,   149,   218,   258,
              264,
            ...
            12750, 12751, 12752, 12757, 12797, 12824, 12887, 12895, 12918,
            12941],
           dtype='int64', length=1188)
Int64Index([    5,     6,    77,   126,   143,   156,   194,   207,   

1

In [293]:
dic_ind = { 0: 0, 1:0, 2:0, 3:0, 4:0, 5:0 }

In [298]:
for i in range(6):
    dic_ind[i] = np.random.choice(dic[i],minority,replace=False)
    

In [300]:
 under_sample_indices = np.concatenate((dic_ind[0],dic_ind[1],dic_ind[2],dic_ind[3],dic_ind[4],dic_ind[5]), axis=0,  )

In [303]:
under_sample_df = iphone_df.loc[under_sample_indices]

,iphone,samsunggalaxy,sonyxperia,nokialumina,htcphone,ios,googleandroid,iphonecampos,samsungcampos,sonycampos,...,sonyperunc,nokiaperunc,htcperunc,iosperpos,googleperpos,iosperneg,googleperneg,iosperunc,googleperunc,iphonesentiment
817,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2767,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4540,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1880,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,2,0,2,0,0
508,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1712,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9267,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5474,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [305]:
X_iphone,y_iphone = X_Y_split(under_sample_df,'iphonesentiment')

X_train_iphone , X_test_iphone , y_train_iphone , y_test_iphone =  train_test_split(X_iphone, y_iphone, test_size=0.33, random_state=42)




In [306]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,cohen_kappa_score
import time

start_time = time.time()

rf = RandomForestClassifier()

rf.fit(X_train_iphone,y_train_iphone)

predictions = rf.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.0 mins--- 
 
Accuracy :  0.47347994825355755


Kappa's :  0.37000814995925024


Confusion Matrix : 
 
   [[95  1  6  2  4  2]
 [ 8 28 17  6 14 19]
 [ 3  7 31  3  5  4]
 [ 0  7  5 82  6  6]
 [ 1  4  6  3 47  8]
 [29 88 58 30 55 83]]


In [307]:
under_sample_df['iphonesentiment'].value_counts()

5    390
3    390
1    390
4    390
2    390
0    390
Name: iphonesentiment, dtype: int64

In [316]:
X_iphone,y_iphone = X_Y_split(under_sample_df,'iphonesentiment')



In [324]:

rf = RandomForestClassifier()

rf.fit(X_iphone,y_iphone)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [325]:
X, y = X_Y_split(iphone_df,'iphonesentiment')

X_train , X_test , y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)



In [326]:
predictions = rf.predict(X_test)

In [327]:
print("Accuracy : ",accuracy_score(predictions,y_test))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test) )


Accuracy :  0.6205044371788884


Kappa's :  0.41694002381929784


Confusion Matrix : 
 
   [[ 396    0    1    4    5   32]
 [  43   38   12   18   45  394]
 [  17    3   58    7   22  184]
 [   9    0    1  283   10   78]
 [  10    1    1    5  197  127]
 [ 140   72   89   82  213 1685]]


In [41]:
iphone_df['iphonesentiment'] =  iphone_df['iphonesentiment'].map({0:1 , 1:1 , 2:2 , 3:2, 4:3, 5:3})

In [46]:
#Create training and testing sets with a 70/30 split using createDataPartition.
from sklearn.model_selection import train_test_split

X_iphone,y_iphone = X_Y_split(iphone_df,'iphonesentiment')

X_train_iphone , X_test_iphone , y_train_iphone , y_test_iphone =  train_test_split(X_iphone, y_iphone, test_size=0.33, random_state=42)




In [51]:
y_test_iphone.value_counts()

y_train_iphone.value_counts()

2    7068
1    1623
Name: iphonesentiment, dtype: int64

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,cohen_kappa_score
import time

start_time = time.time()

rf = RandomForestClassifier()

rf.fit(X_train_iphone,y_train_iphone)

predictions = rf.predict(X_test_iphone)

end_time = start_time - time.time()

print("--- Run Time is : %s mins--- \n " % np.round(((time.time()-start_time)/60),2))
print("Accuracy : ",accuracy_score(predictions,y_test_iphone))
print("\n")

print("Kappa's : ",cohen_kappa_score(predictions,y_test_iphone))

print("\n")
print("Confusion Matrix : \n \n  ",confusion_matrix(predictions,y_test_iphone) )


--- Run Time is : 0.01 mins--- 
 
Accuracy :  0.9152265296590378


Kappa's :  0.6348532921007342


Confusion Matrix : 
 
   [[ 384   18]
 [ 345 3535]]


In [ ]:
#Create training and testing sets with a 70/30 split using createDataPartition.
from sklearn.model_selection import train_test_split

X_iphone,y_iphone = X_Y_split(iphone_df,'iphonesentiment')

X_train_iphone , X_test_iphone , y_train_iphone , y_test_iphone =  train_test_split(X_iphone, y_iphone, test_size=0.33, random_state=42)


